In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [3]:
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

In [4]:
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

In [5]:
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

In [6]:
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [9]:
if __name__ == '__main__':

    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    links_list = []

    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

ConnectionError: HTTPSConnectionPool(host='www.amazon.comhttps', port=443): Max retries exceeded with url: /aax-us-iad.amazon.com/x/c/JC77JXmlpLjXSIt07rKSl9AAAAGQab0VrwEAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICBgfiQ2/https://www.amazon.com/ASFSKY-Storage-Universal-Compatible-Nintendo/dp/B0CZHG6QJJ/ref=sxbs_sbv_search_btf?content-id=amzn1.sym.2f0a8989-0b67-47e7-b61e-9e3ef9908602%3Aamzn1.sym.2f0a8989-0b67-47e7-b61e-9e3ef9908602&cv_ct_cx=playstation+4&dib=eyJ2IjoiMSJ9.EThVg9hSw5EixCcbH23Gng.8iMH7x792-DBIBi-M4WvXthJZpqBk6yxD03JEPjfvoI&dib_tag=se&keywords=playstation+4&pd_rd_i=B0CZHG6QJJ&pd_rd_r=c61847e8-d75c-41f8-a883-0ffb5a14b38b&pd_rd_w=YWdeG&pd_rd_wg=8Ts0X&pf_rd_p=2f0a8989-0b67-47e7-b61e-9e3ef9908602&pf_rd_r=ZQZZ5VY8JWS2ANQKHV0D&qid=1719760917&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-1-a61ee601-6e56-4862-a8a2-1d3da5a5406f (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E7E2626570>: Failed to resolve 'www.amazon.comhttps' ([Errno 11001] geta

In [ ]:
amazon_df

In [11]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import numpy as np
import time

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1'
}

def get_product_info(link, headers):
    try:
        # Check if link is a full URL or a relative path and construct the full URL
        if not link.startswith('http'):
            full_url = urljoin("https://www.amazon.com", link)
        else:
            full_url = link
        
        new_webpage = requests.get(full_url, headers=headers)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        title = get_title(new_soup)
        price = get_price(new_soup)
        rating = get_rating(new_soup)
        reviews = get_review_count(new_soup)
        availability = get_availability(new_soup)

        return title, price, rating, reviews, availability
    except Exception as e:
        print(f"Error processing {link}: {e}")
        return None, None, None, None, None

def get_title(soup):
    try:
        return soup.find("span", attrs={"id": "productTitle"}).text.strip()
    except Exception as e:
        print(f"Error getting title: {e}")
        return ""

def get_price(soup):
    try:
        price_element = soup.find("span", attrs={"id": "priceblock_ourprice"})
        if price_element:
            return price_element.text.strip()
        else:
            return "Price information not available"
    except Exception as e:
        print(f"Error getting price: {e}")
        return ""


def get_rating(soup):
    try:
        return soup.find("span", attrs={"class": "a-icon-alt"}).text.strip().split()[0]
    except Exception as e:
        print(f"Error getting rating: {e}")
        return ""

def get_review_count(soup):
    try:
        return soup.find("span", attrs={"id": "acrCustomerReviewText"}).text.strip().split()[0]
    except Exception as e:
        print(f"Error getting review count: {e}")
        return ""

def get_availability(soup):
    try:
        return soup.find("div", attrs={"id": "availability"}).text.strip()
    except Exception as e:
        print(f"Error getting availability: {e}")
        return ""

if __name__ == '__main__':
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})
    links_list = [link.get('href') for link in links]

    d = {"title":[], "price":[], "rating":[], "reviews":[], "availability":[]}

    for link in links_list:
        title, price, rating, reviews, availability = get_product_info(link, HEADERS)
        d['title'].append(title)
        d['price'].append(price)
        d['rating'].append(rating)
        d['reviews'].append(reviews)
        d['availability'].append(availability)
        time.sleep(2)  # Adding a delay to be respectful of Amazon's servers

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


C:\Users\o876\AppData\Local\Temp\ipykernel_16876\2621135919.py:102: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [12]:
def_amazon

NameError: name 'def_amazon' is not defined

In [13]:
amazon_df

,title,price,rating,reviews,availability
0,PlayStation®4 Console – Call of Duty® Modern W...,Price information not available,4.4,81,In Stock
1,Wireless Controller Dual Vibration Game Joysti...,Price information not available,4.1,"1,941",In Stock
2,"FASIGO PS4 Controller 2 Pack, Wireless PS4 Con...",Price information not available,4.2,28,In Stock
3,DualShock 4 Wireless Controller for PlayStatio...,Price information not available,4.6,"168,067",
4,"PS4 Controller Wireless, with USB C Cable/1000...",Price information not available,4.2,"2,899",In Stock
5,Wireless Controller for PS4 with 2 Thumb Grips...,Price information not available,4.2,"2,440",In Stock
6,Mortal Kombat XL (PS4),Price information not available,4.7,"2,907",In Stock
7,OIVO PS4 Stand Cooling Fan Station for Playsta...,Price information not available,4.5,"45,602",In Stock
8,Minecraft Starter Collection (PS4),Price information not available,4.6,760,Only 10 left in stock - order soon.
9,"KDYGPDCT 2 Pack Wireless Controller for PS4,Du...",Price information not available,4.5,41,In Stock
